<a href="https://colab.research.google.com/github/ashwinvaswani/Generative-Modelling-of-Images-from-Speech/blob/master/src/TIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydub
!pip install pytube
!pip install mtcnn
!pip install keras-vggface

     |████████████████████████████████| 2.3MB 2.8MB/s 
  Created wheel for mtcnn: filename=mtcnn-0.0.9-cp36-none-any.whl size=2257690 sha256=3487265bbcd98f2dca956dd4fdb1ca60eb85caa3765fa85cadbc9ef7d8494c12
  Stored in directory: /root/.cache/pip/wheels/85/81/65/6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
Successfully built mtcnn


In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2 
import os
import sys
import moviepy.editor as mp
import pickle

import math
from math import ceil
from math import floor

from pydub import AudioSegment
from pytube import YouTube

import shutil

import IPython.display as ipd
import librosa
import librosa.display

import dlib

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from numpy import expand_dims
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2048000/45929032 bytes (4.5%)6348800/45929032 bytes (13.8%)10551296/45929032 bytes (23.0%)14794752/45929032 bytes (32.2%)19136512/45929032 bytes (41.7%)23224320/45929032 bytes (50.6%)27500544/45929032 bytes (59.9%)31744000/45929032 bytes (69.1%)36069376/45929032 bytes (78.5%)40452096/45929032 bytes (88.1%)44843008/45929032 bytes (97.6%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
PATH = './drive/My Drive/TIP/Dataset/'
PATH_TO_MAIN = './drive/My Drive/TIP/'
YT_LINK = 'www.youtube.com/watch?v='

In [0]:
column_names = ['YouTube_ID', 'start_segment', 'end_segment', 'X_coordinate', 'Y_coordinate']
train_df = pd.read_csv(PATH + 'avspeech_train.csv',names = column_names)
test_df = pd.read_csv(PATH + 'avspeech_test.csv',names = column_names)

In [6]:
train_df.head()

,YouTube_ID,start_segment,end_segment,X_coordinate,Y_coordinate
0,CJoOwXcjhds,233.266000,239.367000,0.780469,0.670833
1,AvWWVOgaMlk,90.000000,93.566667,0.586719,0.311111
2,Y8HMIm8mdns,171.607767,174.607767,0.505729,0.240741
3,akwvpAiLFk0,144.680000,150.000000,0.698438,0.288889
4,Swss72CHSWg,90.023267,97.297200,0.230729,0.204630


In [7]:
test_df.head()

,YouTube_ID,start_segment,end_segment,X_coordinate,Y_coordinate
0,u5MPyrRJPmc,108.2400,111.240000,0.849219,0.305556
1,H1ulMfj5wRY,112.3200,116.940000,0.112500,0.345833
2,-wuxbgMRIWs,30.0300,36.745044,0.744531,0.211111
3,GNRPRH-E-sI,30.2302,38.171467,0.333594,0.494444
4,VvcwAGkSy2o,240.2000,253.366667,0.491667,0.372222


In [8]:
YT_LINK + train_df.iloc[2]["YouTube_ID"]

'www.youtube.com/watch?v=Y8HMIm8mdns'

In [9]:

# shutil.rmtree('Images')
# shutil.rmtree('Audio')
# shutil.rmtree('Videos')
os.mkdir("Images")
os.mkdir("Audio")
os.mkdir("Videos")

len_train = train_df.shape[0]
count_unsuccessful = 0
dict_audio = {}
to_remove = []
for i in range(1,50):

  print(YT_LINK + train_df.iloc[i]["YouTube_ID"])
  try:
    yt = YouTube(YT_LINK + train_df.iloc[i]["YouTube_ID"])
    stream = yt.streams.first()
    stream.download(output_path = 'Videos/',filename = train_df.iloc[i]["YouTube_ID"])


    cap = cv2.VideoCapture("Videos/" + train_df.iloc[i]["YouTube_ID"] + '.mp4') 
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    if fps == 0:
      continue

    count = 0

    dir_name = train_df.iloc[i]["YouTube_ID"]
    
    print("FPS is : "  + str(fps))
    start_cnt = train_df.iloc[i]["start_segment"]
    end_cnt = train_df.iloc[i]["end_segment"]
    
    while(1):

        ret,frame = cap.read()

        if ret == 1:

          count += 1

          (h, w) = frame.shape[:2]

          if (((start_cnt + end_cnt)*fps)/2) - 16 < count < (((start_cnt + end_cnt)*fps)/2) + 16:
            dnnFaceDetector = dlib.cnn_face_detection_model_v1( PATH_TO_MAIN + "Face_detection/mmod_human_face_detector.dat")
            faceRects = dnnFaceDetector(frame, 0)
            print(faceRects)

            if len(faceRect) == 0:
              to_remove.append(dir_name)
              continue
            for faceRect in faceRects:
                x1 = faceRect.rect.left()
                y1 = faceRect.rect.top()
                x2 = faceRect.rect.right()
                y2 = faceRect.rect.bottom()
                img = frame[y1:y2, x1:x2]
                print("About to crop")
                print(count)
                img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
                cv2.imwrite('Images/' +dir_name + '.png', img)
            break


          if count > int(end_cnt*fps):
            break
          k = cv2.waitKey(1)
          if k == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
  except:
    print("SORRY!!! The video is unavailable")
    count_unsuccessful += 1
    
  try:
    clip = mp.VideoFileClip("Videos/" + train_df.iloc[i]["YouTube_ID"] + '.mp4').subclip(start_cnt,end_cnt)
    clip.audio.write_audiofile("Audio/" + train_df.iloc[i]["YouTube_ID"] +'.mp3') 
    print("before dict")
    dict_audio[train_df.iloc[i]["YouTube_ID"]] = end_cnt - start_cnt
  except:
    print("Error with audio")  

www.youtube.com/watch?v=AvWWVOgaMlk
FPS is : 30.000015705169986
mmod_rectangles[[(532, 80) (871, 420)]]
About to crop
2738
[MoviePy] Writing audio in Audio/AvWWVOgaMlk.mp3


100%|██████████| 79/79 [00:00<00:00, 592.49it/s]

[MoviePy] Done.
before dict
www.youtube.com/watch?v=Y8HMIm8mdns


FPS is : 29.97002997002997
mmod_rectangles[[(574, 114) (710, 251)]]
About to crop
5173
[MoviePy] Writing audio in Audio/Y8HMIm8mdns.mp3


100%|██████████| 67/67 [00:00<00:00, 592.52it/s]

...

[MoviePy] Done.


before dict
www.youtube.com/watch?v=qzM4wshoqGs
FPS is : 25.0
mmod_rectangles[[(428, 212) (507, 291)], [(668, 220) (747, 299)]]
About to crop
2306
About to crop
2306
[MoviePy] Writing audio in Audio/qzM4wshoqGs.mp3


100%|██████████| 67/67 [00:00<00:00, 548.36it/s]

[MoviePy] Done.
before dict
www.youtube.com/watch?v=871zAw-g1ZM


SORRY!!! The video is unavailable
Error with audio


In [0]:
# Preparing y_train for encoder and x_train for decoder

In [0]:
def get_embeddings(filenames):

  faces = []
  for f in filenames:
    img = cv2.imread(f)
    face_arr = asarray(img)
    faces.append(face_arr)

  samples = asarray(faces,'float32')

  samples = preprocess_input(samples,version=2)

  model = VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3),pooling='avg')

  yhat = model.predict(samples)

  return yhat

In [12]:
face_list = []
for elem in os.listdir('./Images'):
  face_list.append('./Images/' + elem)
y_train_encoder = get_embeddings(face_list)
x_train_decoder = get_embeddings(face_list)














94699520/94694792 [==============================] - 1s 0us/step


In [13]:
y_train_encoder.shape

(42, 2048)

In [0]:
# Using y_train to create decoder

In [0]:
# Input is y_train and output is images from Images folder

In [0]:
# Augmenting Audio

In [52]:
shutil.rmtree('Processed_Audio')
os.mkdir('Processed_Audio')
temp_dict = dict_audio.copy()
for elem in os.listdir('./Audio'):
  sound = AudioSegment.from_mp3('./Audio/' + elem)
  sound_new = sound
  while temp_dict[elem[:-4]] < 6:
    print("here")
    sound_new += sound
    temp = temp_dict[elem[:-4]]
    temp_dict[elem[:-4]] = 2*temp
    sound = sound_new

  extract = sound[0:6*1000]

  extract.export('Processed_Audio/' + elem, format="mp3")


here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


In [0]:
to_remove = ['2f32XSMYlDk']
for elem in to_remove:
  os.remove('Processed_Audio/' + elem + '.mp3')

In [0]:
# Preparing x_train

In [57]:
shutil.rmtree('Spectograms')
os.mkdir('Spectograms')
for filename in os.listdir("./Processed_Audio"):
  audio_path = './Processed_Audio/' + filename
  x , sr = librosa.load(audio_path,sr = 15925)
  print(audio_path)
  print(type(x), type(sr))
  print(x.shape, sr)
  ipd.Audio(audio_path)
  window_size = 25
  window = np.hanning(window_size)
  h1 = 160
  print(h1)

  stft  = librosa.core.spectrum.stft(x, n_fft=512, hop_length=h1,win_length=window_size, window=window)
  out = 2 * np.abs(stft) / np.sum(window_size)
  with open('./Spectograms/' + filename[:-4] + '.pkl','wb') as f:
    pickle.dump(stft,f)
  
  # librosa.display.waveplot(x, sr=sr)
  # mfccs = librosa.feature.mfcc(x, sr=sr)
  # print(mfccs.shape)
  # librosa.display.specshow(mfccs, sr=sr, x_axis='time')
  # print()




./Processed_Audio/labiHToR5nk.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/DIWf1t-HzwI.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/h-fSfAFufCo.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/bNxD_breZy8.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/tdTXVU5wN8I.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/7rYeSDHS0U0.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/sujFCXbYkMo.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/ymD5uLlLc0g.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/YXcVkIEMGds.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/UBL2Vowiulk.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/Y8HMIm8mdns.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160

In [0]:
# Todo: For all pickle files in spectogram 

In [0]:
def get_encoder_network_input(k):
  mod = np.zeros((257,598))
  for i in range(257):
    for j in range(598):
      mod[i][j] = np.abs(k[i][j])

  theta_ = np.zeros((257,598))
  for i in range(257):
    for j in range(598):
      theta_[i][j] = np.angle(k[i][j])

  real = np.zeros((257,598))
  complex_ = np.zeros((257,598))
  for i in range(257):
    for j in range(598):
      temp = mod[i][j]
      real[i][j] = temp * math.cos(complex_[i][j])
      complex_[i][j] = temp * math.sin(complex_[i][j])

  real = np.expand_dims(real,axis = 0)
  complex_ = np.expand_dims(complex_,axis = 0)
  combined = np.concatenate((real,complex_))

  return combined

In [59]:
encoder_input = []
for elem in os.listdir('./Spectograms/'):
  with open('./Spectograms/' + elem,'rb') as f:
    print('./Spectograms/' + elem)
    k = pickle.load(f)
    print(k.shape)

  encoder_input.append(np.transpose(get_encoder_network_input(k)))

./Spectograms/labiHToR5nk.pkl
(257, 598)
./Spectograms/zFH0QbS-l-w.pkl
(257, 598)
./Spectograms/h5wT_c4fQ1o.pkl
(257, 598)
./Spectograms/IrXrbrZWflA.pkl
(257, 598)
./Spectograms/lcClO5lHEjA.pkl
(257, 598)
./Spectograms/uCGKDCWxqxo.pkl
(257, 598)
./Spectograms/rCp8Jae81KU.pkl
(257, 598)
./Spectograms/JjduaMIoKvI.pkl
(257, 598)
./Spectograms/7rYeSDHS0U0.pkl
(257, 598)
./Spectograms/qzM4wshoqGs.pkl
(257, 598)
./Spectograms/aaEA__Js2u0.pkl
(257, 598)
./Spectograms/8nQBG5hvjpk.pkl
(257, 598)
./Spectograms/xbgfxIc-nbs.pkl
(257, 598)
./Spectograms/sujFCXbYkMo.pkl
(257, 598)
./Spectograms/YXcVkIEMGds.pkl
(257, 598)
./Spectograms/AvWWVOgaMlk.pkl
(257, 598)
./Spectograms/tdTXVU5wN8I.pkl
(257, 598)
./Spectograms/UBL2Vowiulk.pkl
(257, 598)
./Spectograms/BjvtZkHWExY.pkl
(257, 598)
./Spectograms/Cy9SUMj5wGY.pkl
(257, 598)
./Spectograms/R0u9E8GsUXk.pkl
(257, 598)
./Spectograms/PmD-LzPS2rg.pkl
(257, 598)
./Spectograms/QoQF8N5ZsQA.pkl
(257, 598)
./Spectograms/3AsPqH3QaQM.pkl
(257, 598)
./Spectograms/8E

In [60]:
len(encoder_input)

42

In [0]:
# For encoder

# Input : encoder input as np array
# Output : y_train_encoder

# FOr decoder

# Input : x_train_encoder
# Output : Images from Images folder

In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D,AvgPool2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,BatchNormalization,LeakyReLU,MaxPool2D
from keras.models import Sequential
from keras.layers import Conv2DTranspose,Reshape

In [0]:
# TODO: create decoder output into np array from images folder

In [0]:
# ENCODER MODEL

In [0]:
x_enc_train = np.asarray(encoder_input)

In [62]:
x_enc_train.shape

(42, 598, 257, 2)

In [63]:
y_train_encoder.shape

(42, 2048)

In [0]:
with open('encoder_trainX.pkl','wb') as f:
    pickle.dump(x_enc_train,f)

In [0]:
with open('encoder_trainY.pkl','wb') as f:
    pickle.dump(y_train_encoder,f)

In [0]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(x_enc_train,y_train_encoder,test_size = 0.1)

In [69]:
X_train.shape

(37, 598, 257, 2)

In [0]:
from keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
  
def custom_loss(y_pred, y_true):
  y_pred= tf.cast(y_pred, tf.float64) 
  y_true= tf.cast(y_true, tf.float64)
  y_pred=tf.nn.relu(y_pred) 
  return tf.sqrt(tf.reduce_mean(tf.squared_difference(tf.log1p(y_pred), tf.log1p(y_true))))

In [70]:
# input = tf.keras.layers.Input(shape=x_enc_train.shape)
# x = tf.keras.layers.Conv2D(64, kernel_size = (4,4), strides=(1,1), padding = 'same')(input)

model_encoder = Sequential()
model_encoder.add(Conv2D(64, kernel_size=(4,4),strides=(1,1), padding='same',  input_shape=[598,257,2],use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(Conv2D(64, kernel_size=(4,4),strides=(1,1), padding='same', use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(Conv2D(128, kernel_size=(4,4),strides=(1,1), padding='same', use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(MaxPool2D(pool_size=(2,1)))

model_encoder.add(Conv2D(128, kernel_size=(4,4),strides=(1,1), padding='same', use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(MaxPool2D(pool_size=(2,1)))

model_encoder.add(Conv2D(256, kernel_size=(4,4),strides=(1,1), padding='same', use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(MaxPool2D(pool_size=(2,1)))

model_encoder.add(Conv2D(512, kernel_size=(4,4),strides=(1,1), padding='same', use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())
 
model_encoder.add(Conv2D(512, kernel_size=(4,4),strides=(1,1), padding='same', use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(Conv2D(512, kernel_size=(4,4),strides=(1,1), padding='same', use_bias=False))

model_encoder.add(AvgPool2D(strides = (1,1)))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(Flatten())
model_encoder.add(Dense(4096,activation='relu'))
model_encoder.add(Dropout(0.2))

model_encoder.add(Dense(2048,activation='relu'))

model_encoder.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 598, 257, 64)      2048      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 598, 257, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 598, 257, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 598, 257, 64)      65536     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 598, 257, 64)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 598, 257, 64)      256       
_____

In [71]:
model_encoder.compile(loss='mse',optimizer=Adam(lr=1e-3),metrics=['accuracy'])

In [0]:
checkpoint_path = "./cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
 
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    tf.keras.callbacks.EarlyStopping(patience=7, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    cp_callback
]
 
 
model_encoder.fit(X_train, y_train, batch_size=32,validation_data=(X_val, y_val), epochs=10 ,verbose=1)



Train on 37 samples, validate on 5 samples
Epoch 1/10


In [0]:
# DECODER MODEL

In [36]:
filenames = []
for elem in os.listdir('./Images'):
  filenames.append(elem)

decoder_train_df = pd.DataFrame(filenames,columns = ['filename'])
decoder_train_df.head()

,filename
0,Y8HMIm8mdns.png
1,AvWWVOgaMlk.png
2,akwvpAiLFk0.png


In [37]:
model_decoder = Sequential()
model_decoder.add(Dense(14*14*256,input_shape = [2048],activation='relu'))
if K.image_data_format() == 'channels_first':
      model_decoder.add(Reshape((256, 14, 14), input_shape=(256 * 14 * 14,)))
      bn_axis = 1
else:
      model_decoder.add(Reshape((14, 14, 256), input_shape=(256 * 14 * 14,)))
      bn_axis = -1
model_decoder.add(Conv2DTranspose(128,strides=2,kernel_size=2))
model_decoder.add(Conv2DTranspose(64,strides=2,kernel_size=2))
model_decoder.add(Conv2DTranspose(32,strides=2,kernel_size=2))
model_decoder.add(Conv2DTranspose(32,strides=2,kernel_size=2))

model_decoder.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 50176)             102810624 
_________________________________________________________________
reshape_1 (Reshape)          (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 128)       131200    
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 56, 56, 64)        32832     
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 112, 112, 32)      8224      
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 224, 224, 32)      4128      
Total params: 102,987,008
Trainable params: 102,987,008
Non-trainable params: 0
________________________________________

In [0]:
model_decoder.compile(loss='mse',optimizer=Adam(lr=1e-3),metrics=['accuracy'])

In [0]:
checkpoint_path = "./cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
 
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    tf.keras.callbacks.EarlyStopping(patience=7, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    cp_callback
]
 
 
# model_decoder.fit(trainX, trainY, batch_size=1,validation_data=(valX, valY), epochs=200 ,verbose=1)

In [0]:
from google.colab import files
files.download('encoder_trainY.pkl')